In [ ]:
import pandas as pd
import re

# === SETUP ===
META_PATH = "/content/drive/MyDrive/Patent_Data/g_patent.tsv.zip"
CPC_PATH = "/content/drive/MyDrive/Patent_Data/g_cpc_current.tsv.zip"
INV_PATH = "/content/drive/MyDrive/Patent_Data/g_inventor_disambiguated.tsv.zip"
ASS_PATH = "/content/drive/MyDrive/Patent_Data/g_assignee_disambiguated.tsv.zip"
LOC_PATH = "/content/drive/MyDrive/Patent_Data/g_location_disambiguated.tsv.zip"

OUT_PATH = "/content/drive/MyDrive/Patent_Data/patents_2024_enriched.csv"

# --- Domain filters (keywords to keep) ---
# More precise, tech-focused filters
KEYWORDS = [
    "machine learning",
    "deep learning",
    "artificial intelligence",
    "neural network",
    "data pipeline",
    "data processing",
    "data management",
    "data storage",
    "software system",
    "computer system",
    "cloud computing",
    "distributed system",
    "reinforcement learning",
    "graph neural network",
    "data engineering",
    "algorithm optimization",
    "information retrieval",
    "recommendation system",
    "knowledge graph",
    "database system",
    "computational method"
]

YEAR = "2024"

import pandas as pd

CPC_PATH = "/content/drive/MyDrive/Patent_Data/g_cpc_current.tsv.zip"
TECH_PREFIXES = ("G06", "H04", "G16", "G10")  # core tech CPC prefixes

print("🔍 Collecting tech-related CPC patent IDs (optimized)…")

cpc_ids_from_tech_prefixes = set()
counts = {p: 0 for p in TECH_PREFIXES}

# process smaller chunks
for chunk in pd.read_csv(
    CPC_PATH,
    sep="\t",
    compression="zip",
    usecols=["patent_id", "cpc_class"],
    chunksize=100_000,
    dtype=str,
    low_memory=False
):
    chunk.dropna(subset=["cpc_class"], inplace=True)
    chunk["cpc_class"] = chunk["cpc_class"].astype(str)

    # use vectorized .str.startswith with tuple
    mask = chunk["cpc_class"].str.startswith(TECH_PREFIXES)
    filtered = chunk[mask]

    # record counts
    for p in TECH_PREFIXES:
        counts[p] += int(filtered["cpc_class"].str.startswith(p).sum())

    cpc_ids_from_tech_prefixes.update(filtered["patent_id"].unique())

print(f"✅ Found {len(cpc_ids_from_tech_prefixes):,} tech-related patents")
print("   Breakdown:", counts)








🔍 Collecting tech-related CPC patent IDs (optimized)…
✅ Found 2,240,627 tech-related patents
   Breakdown: {'G06': 5140656, 'H04': 6319113, 'G16': 181256, 'G10': 257165}


In [ ]:
import pandas as pd

CPC_PATH = "/content/drive/MyDrive/Patent_Data/g_cpc_current.tsv.zip"
sample = pd.read_csv(CPC_PATH, sep="\t", compression="zip", nrows=5)
print("Columns:", list(sample.columns))
print(sample.head())


Columns: ['patent_id', 'cpc_sequence', 'cpc_section', 'cpc_class', 'cpc_subclass', 'cpc_group', 'cpc_type']
   patent_id  cpc_sequence cpc_section cpc_class cpc_subclass  cpc_group  \
0    3950000             0           A       A63         A63C  A63C9/001   
1    3950000             1           A       A63         A63C   A63C9/00   
2    3950000             2           A       A63         A63C  A63C9/002   
3    3950000             3           A       A63         A63C  A63C9/081   
4    3950001             0           A       A63         A63C  A63C9/086   

      cpc_type  
0  inventional  
1  inventional  
2  inventional  
3  inventional  
4  inventional  


In [ ]:
import pandas as pd

CPC_PATH = "/content/drive/MyDrive/Patent_Data/g_cpc_current.tsv.zip"
TECH_PREFIXES = ("G06", "H04", "G16", "G10")

out_path = "/content/drive/MyDrive/Patent_Data/tech_patent_ids_2024.parquet"
writer = pd.DataFrame(columns=["patent_id"])  # placeholder

all_ids = set()
for chunk in pd.read_csv(
    CPC_PATH,
    sep="\t",
    compression="zip",
    usecols=["patent_id", "cpc_class"],
    chunksize=100_000,
    dtype=str,
):
    chunk.dropna(subset=["cpc_class"], inplace=True)
    mask = chunk["cpc_class"].str.startswith(TECH_PREFIXES)
    sub = chunk.loc[mask, ["patent_id"]]
    all_ids.update(sub["patent_id"].unique())

# Save as a compact Parquet file
pd.DataFrame({"patent_id": list(all_ids)}).to_parquet(out_path, index=False)
print(f"✅ Saved {len(all_ids):,} CPC IDs to {out_path}")


✅ Saved 2,240,627 CPC IDs to /content/drive/MyDrive/Patent_Data/tech_patent_ids_2024.parquet


In [ ]:
import pandas as pd
import pyarrow as pa
import pyarrow.parquet as pq
from tqdm import tqdm

PATENT_PATH = "/content/drive/MyDrive/Patent_Data/g_patent.tsv.zip"
ABSTRACT_PATH = "/content/drive/MyDrive/Patent_Data/g_patent_abstract.tsv.zip"
core_path = "/content/drive/MyDrive/Patent_Data/patents_2024_core.parquet"

chunksize = 50_000
writer = None

reader = pd.read_csv(
    PATENT_PATH,
    sep="\t",
    compression="zip",
    usecols=["patent_id", "patent_date", "patent_title"],
    dtype=str,
    chunksize=chunksize,
    iterator=True,
)

for chunk in tqdm(reader, desc="Processing patents"):
    chunk = chunk[chunk["patent_date"].fillna("").str.startswith("2024")]
    if chunk.empty:
        continue

    # Reset index so "__index_level_0__" is not accidentally written
    chunk = chunk.reset_index(drop=True)

    # Convert to Arrow Table
    table = pa.Table.from_pandas(chunk, preserve_index=False)

    # Create writer if first chunk
    if writer is None:
        writer = pq.ParquetWriter(core_path, table.schema)
    writer.write_table(table)

if writer:
    writer.close()

print(f"✅ Metadata written to {core_path}")





Processing patents: 186it [00:30,  6.02it/s]


✅ Metadata written to /content/drive/MyDrive/Patent_Data/patents_2024_core.parquet


In [ ]:
import pandas as pd
import pyarrow as pa
import pyarrow.parquet as pq
from tqdm import tqdm

core_path = "/content/drive/MyDrive/Patent_Data/patents_2024_core.parquet"
ABSTRACT_PATH = "/content/drive/MyDrive/Patent_Data/g_patent_abstract.tsv.zip"
merged_path = "/content/drive/MyDrive/Patent_Data/patents_2024_core_final.parquet"

meta = pd.read_parquet(core_path)
meta_ids = set(meta["patent_id"].astype(str))

writer = None
chunksize = 100_000

reader = pd.read_csv(
    ABSTRACT_PATH,
    sep="\t",
    compression="zip",
    usecols=["patent_id", "patent_abstract"],
    dtype=str,
    chunksize=chunksize,
    iterator=True,
)

for chunk in tqdm(reader, desc="Merging abstracts"):
    # Keep only rows with patent_ids we actually need
    chunk = chunk[chunk["patent_id"].isin(meta_ids)]
    if chunk.empty:
        continue

    # Merge just this chunk
    merged_chunk = meta.merge(chunk, on="patent_id", how="left")
    merged_chunk["patent_abstract"] = merged_chunk["patent_abstract"].fillna("Not available")
    merged_chunk = merged_chunk.reset_index(drop=True)

    # Convert and append
    table = pa.Table.from_pandas(merged_chunk, preserve_index=False)
    if writer is None:
        writer = pq.ParquetWriter(merged_path, table.schema)
    writer.write_table(table)

if writer:
    writer.close()

print(f"✅ Saved merged dataset → {merged_path}")


Merging abstracts: 93it [02:47,  1.80s/it]

✅ Saved merged dataset → /content/drive/MyDrive/Patent_Data/patents_2024_core_final.parquet


In [ ]:
import pyarrow.parquet as pq

core_path = "/content/drive/MyDrive/Patent_Data/patents_2024_core_final.parquet"
tech_ids_path = "/content/drive/MyDrive/Patent_Data/tech_patent_ids_2024.parquet"

core = pq.read_table(core_path).to_pandas()
tech_ids = set(pq.read_table(tech_ids_path).to_pandas()["patent_id"])

tech_patents = core[core["patent_id"].isin(tech_ids)].reset_index(drop=True)
print(f"✅ Filtered {len(tech_patents):,} tech patents from 2024")

tech_patents_path = "/content/drive/MyDrive/Patent_Data/patents_2024_tech.parquet"
tech_patents.to_parquet(tech_patents_path, index=False)
print(f"💾 Saved tech patents → {tech_patents_path}")


✅ Filtered 904,477 tech patents from 2024
💾 Saved tech patents → /content/drive/MyDrive/Patent_Data/patents_2024_tech.parquet


In [ ]:
import pandas as pd
import pyarrow as pa
import pyarrow.parquet as pq
from tqdm import tqdm

# --- Base paths ---
BASE = "/content/drive/MyDrive/Patent_Data"
core_path = f"{BASE}/patents_2024_core_final.parquet"
out_path = f"{BASE}/patents_2024_enriched.parquet"

# --- Load the base patent data (we only keep IDs for filtering) ---
core = pd.read_parquet(core_path, columns=["patent_id"])
core_ids = set(core["patent_id"].astype(str))
print(f"✅ Loaded {len(core_ids):,} base 2024 patent IDs")

# --- Define helpers for chunk-safe parquet writing ---
def init_writer(schema, path):
    return pq.ParquetWriter(path, schema)

def append_chunk(df, writer):
    table = pa.Table.from_pandas(df, preserve_index=False)
    writer.write_table(table)

# --- Prepare CPC data ---
cpc_path = f"{BASE}/g_cpc_current.tsv.zip"
cpc_cols = ["patent_id","cpc_section","cpc_class","cpc_subclass","cpc_group","cpc_type"]
cpc_chunks = pd.read_csv(cpc_path, sep="\t", compression="zip", usecols=cpc_cols, dtype=str, chunksize=100_000)
cpc_filtered = []

for chunk in tqdm(cpc_chunks, desc="Filtering CPC"):
    chunk = chunk[chunk["patent_id"].isin(core_ids)]
    if not chunk.empty:
        cpc_filtered.append(chunk)

cpc = pd.concat(cpc_filtered, ignore_index=True)
cpc = cpc.groupby("patent_id").agg({
    "cpc_section": "; ".join,
    "cpc_class": "; ".join,
    "cpc_subclass": "; ".join,
    "cpc_group": "; ".join,
    "cpc_type": "; ".join
}).reset_index()
print(f"✅ CPC merged → {len(cpc):,} rows")




✅ Loaded 373,852 base 2024 patent IDs


Filtering CPC: 572it [04:36,  2.07it/s]


✅ CPC merged → 324,037 rows


Filtering Inventors: 235it [02:47,  1.40it/s]


TypeError: sequence item 0: expected str instance, float found

In [ ]:
# --- Inventors ---
inv_path = f"{BASE}/g_inventor_disambiguated.tsv.zip"
inv_cols = ["patent_id","inventor_sequence","gender_code","location_id",
            "disambig_inventor_name_first","disambig_inventor_name_last"]
inv_chunks = pd.read_csv(inv_path, sep="\t", compression="zip",
                         usecols=inv_cols, dtype=str, chunksize=100_000)
inv_filtered = []

for chunk in tqdm(inv_chunks, desc="Filtering Inventors"):
    chunk = chunk[chunk["patent_id"].isin(core_ids)]
    if not chunk.empty:
        inv_filtered.append(chunk)

inv = pd.concat(inv_filtered, ignore_index=True)

# 👇 Clean before grouping — fill NaN and ensure string type
for col in ["disambig_inventor_name_first", "disambig_inventor_name_last", "gender_code", "location_id"]:
    inv[col] = inv[col].fillna("").astype(str)

inv = inv.groupby("patent_id").agg({
    "disambig_inventor_name_first": lambda x: "; ".join([v for v in x if v]),
    "disambig_inventor_name_last":  lambda x: "; ".join([v for v in x if v]),
    "gender_code":                  lambda x: "; ".join([v for v in x if v]),
    "location_id":                  lambda x: "; ".join([v for v in x if v])
}).reset_index()

print(f"✅ Inventors merged → {len(inv):,} rows")


Filtering Inventors: 235it [02:43,  1.44it/s]


✅ Inventors merged → 373,852 rows


In [ ]:

# --- Assignees ---
ass_path = f"{BASE}/g_assignee_disambiguated.tsv.zip"
ass_cols = ["patent_id","assignee_sequence","assignee_id","disambig_assignee_individual_name_first",
            "disambig_assignee_individual_name_last","disambig_assignee_organization",
            "assignee_type","location_id"]
ass_chunks = pd.read_csv(ass_path, sep="\t", compression="zip", usecols=ass_cols, dtype=str, chunksize=100_000)
ass_filtered = []

for chunk in tqdm(ass_chunks, desc="Filtering Assignees"):
    chunk = chunk[chunk["patent_id"].isin(core_ids)]
    if not chunk.empty:
        ass_filtered.append(chunk)

ass = pd.concat(ass_filtered, ignore_index=True)

for col in ["assignee_sequence","assignee_id","disambig_assignee_individual_name_first",
            "disambig_assignee_individual_name_last","disambig_assignee_organization",
            "assignee_type","location_id"]:
    ass[col] = ass[col].fillna("").astype(str)


ass = ass.groupby("patent_id").agg({
    "disambig_assignee_individual_name_first": "; ".join,
    "disambig_assignee_individual_name_last": "; ".join,
    "disambig_assignee_organization": "; ".join,
    "assignee_type": "; ".join,
    "location_id": "; ".join
}).reset_index()
print(f"✅ Assignees merged → {len(ass):,} rows")



Filtering Assignees: 86it [01:08,  1.26it/s]


✅ Assignees merged → 344,132 rows


In [ ]:
import pandas as pd
from tqdm import tqdm
import pyarrow.parquet as pq

BASE = "/content/drive/MyDrive/Patent_Data"

# 1️⃣ Collect location IDs actually used by assignees
assignee_ids = set()
for chunk in pd.read_csv(f"{BASE}/g_assignee_disambiguated.tsv.zip",
                         sep="\t", compression="zip", dtype=str,
                         usecols=["location_id"], chunksize=100_000):
    assignee_ids.update(chunk["location_id"].dropna().unique())

print(f"📍 Unique assignee location IDs to keep: {len(assignee_ids):,}")

# 2️⃣ Stream filter g_location_disambiguated.tsv.zip
loc_cols = ["location_id","disambig_city","disambig_state","disambig_country"]
filtered_chunks = []

for chunk in tqdm(pd.read_csv(f"{BASE}/g_location_disambiguated.tsv.zip",
                              sep="\t", compression="zip", dtype=str,
                              usecols=loc_cols, chunksize=100_000),
                  desc="Filtering location chunks"):
    subset = chunk[chunk["location_id"].isin(assignee_ids)]
    if not subset.empty:
        filtered_chunks.append(subset)

loc_filtered = pd.concat(filtered_chunks, ignore_index=True)
print(f"✅ Filtered locations: {len(loc_filtered):,} rows")

# 3️⃣ Save filtered data
out_path = f"{BASE}/filtered_assignee_locations_2024.parquet"
loc_filtered.to_parquet(out_path, index=False)
print(f"💾 Saved reduced location dataset → {out_path}")


📍 Unique assignee location IDs to keep: 38,241


Filtering location chunks: 1it [00:00,  3.32it/s]

✅ Filtered locations: 38,241 rows
💾 Saved reduced location dataset → /content/drive/MyDrive/Patent_Data/filtered_assignee_locations_2024.parquet


In [ ]:
import pandas as pd
from tqdm import tqdm

BASE = "/content/drive/MyDrive/Patent_Data"

# --- Load prepared data ---
core_path = f"{BASE}/patents_2024_core_final.parquet"
loc_path  = f"{BASE}/filtered_assignee_locations_2024.parquet"

meta = pd.read_parquet(core_path)
loc = pd.read_parquet(loc_path)

print(f"Loaded: {len(meta):,} patents | {len(cpc):,} CPC | {len(inv):,} inventors | {len(ass):,} assignees | {len(loc):,} locations")




Loaded: 2,616,964 patents | 324,037 CPC | 373,852 inventors | 344,132 assignees | 38,241 locations


In [ ]:
# --- Merge CPC, inventor, assignee first ---
merged = (
    meta.merge(cpc, on="patent_id", how="left")
        .merge(inv, on="patent_id", how="left")
        .merge(ass, on="patent_id", how="left")
)

# --- Merge location info via assignee location_id only ---
print("Merging filtered locations with assignee data...")
merged = merged.merge(
    loc[["location_id", "disambig_city", "disambig_state", "disambig_country"]],
    left_on="location_id_y", right_on="location_id", how="left"
)

# --- Combine readable location text ---
merged["assignee_location_info"] = (
    merged[["disambig_city", "disambig_state", "disambig_country"]]
    .fillna("")
    .agg(", ".join, axis=1)
    .str.strip(", ")
)

# Drop helper columns
merged = merged.drop(columns=["location_id_x", "location_id_y", "location_id",
                              "disambig_city", "disambig_state", "disambig_country"],
                     errors="ignore")

# --- Save final enriched ---
out_path = f"{BASE}/patents_2024_enriched_with_locations.parquet"
merged.to_parquet(out_path, index=False)
print(f"✅ Final enriched dataset saved → {out_path} ({len(merged):,} rows)")

Merging filtered locations with assignee data...
✅ Final enriched dataset saved → /content/drive/MyDrive/Patent_Data/patents_2024_enriched_with_locations.parquet (2,616,964 rows)


In [ ]:
import pandas as pd
import os

BASE = "/content/drive/MyDrive/Patent_Data"
OUT  = f"{BASE}/neo4j_exports_2024"
os.makedirs(OUT, exist_ok=True)

# Load enriched dataset
df = pd.read_parquet(f"{BASE}/patents_2024_tech_final.parquet")

print(f"Loaded enriched data → {len(df):,} patents")

# --- PATENT NODES ---
patent_nodes = df[["patent_id", "patent_title", "patent_abstract", "patent_date"]].drop_duplicates()
patent_nodes["year"] = patent_nodes["patent_date"].str[:4]
patent_nodes.rename(columns={
    "patent_id": "id",
    "patent_title": "title",
    "patent_abstract": "abstract"
}, inplace=True)
patent_nodes.to_csv(f"{OUT}/patent_nodes.csv", index=False)
print(f"🟢 Patent nodes: {len(patent_nodes):,}")

# --- INVENTOR NODES ---
if "inventor_id" in df.columns:
    inventor_nodes = df[["inventor_id", "disambig_inventor_name_first", "disambig_inventor_name_last", "gender_code"]].dropna(subset=["inventor_id"]).drop_duplicates()
    inventor_nodes["name"] = inventor_nodes[["disambig_inventor_name_first", "disambig_inventor_name_last"]].fillna("").agg(" ".join, axis=1).str.strip()
    inventor_nodes = inventor_nodes[["inventor_id", "name", "gender_code"]]
    inventor_nodes.rename(columns={"inventor_id": "id"}, inplace=True)
    inventor_nodes.to_csv(f"{OUT}/inventor_nodes.csv", index=False)
    print(f"🟢 Inventor nodes: {len(inventor_nodes):,}")

# --- ASSIGNEE NODES ---
if "assignee_id" in df.columns:
    assignee_nodes = df[["assignee_id", "organization"]].dropna(subset=["assignee_id"]).drop_duplicates()
    assignee_nodes.rename(columns={"assignee_id": "id", "organization": "name"}, inplace=True)
    assignee_nodes.to_csv(f"{OUT}/assignee_nodes.csv", index=False)
    print(f"🟢 Assignee nodes: {len(assignee_nodes):,}")

# --- CPC NODES ---
if "cpc_class" in df.columns:
    cpc_nodes = df[["cpc_class", "cpc_subclass", "cpc_group", "cpc_type"]].drop_duplicates()
    cpc_nodes.rename(columns={"cpc_class": "id", "cpc_subclass": "subclass", "cpc_group": "group", "cpc_type": "type"}, inplace=True)
    cpc_nodes.to_csv(f"{OUT}/cpc_nodes.csv", index=False)
    print(f"🟢 CPC nodes: {len(cpc_nodes):,}")

# --- LOCATION NODES ---
if "assignee_location_info" in df.columns:
    loc_nodes = df[["assignee_location_info"]].dropna().drop_duplicates()
    loc_nodes["id"] = loc_nodes["assignee_location_info"].astype("category").cat.codes
    loc_nodes = loc_nodes[["id", "assignee_location_info"]]
    loc_nodes.rename(columns={"assignee_location_info": "location"}, inplace=True)
    loc_nodes.to_csv(f"{OUT}/location_nodes.csv", index=False)
    print(f"🟢 Location nodes: {len(loc_nodes):,}")


Loaded enriched data → 54,400 patents
🟢 Patent nodes: 22,282
🟢 CPC nodes: 15,215
🟢 Location nodes: 1,257


In [ ]:
df.columns

Index(['patent_id', 'patent_date', 'patent_title', 'patent_abstract',
       'cpc_section', 'cpc_class', 'cpc_subclass', 'cpc_group', 'cpc_type',
       'disambig_inventor_name_first', 'disambig_inventor_name_last',
       'gender_code', 'disambig_assignee_individual_name_first',
       'disambig_assignee_individual_name_last',
       'disambig_assignee_organization', 'assignee_type',
       'assignee_location_info', 'inventor_name'],
      dtype='object')

In [ ]:
# --- RELATIONSHIPS ---

rels = []

# Patent → CPC
if "cpc_class" in df.columns:
    patent_cpc = df[["patent_id", "cpc_class"]].dropna().drop_duplicates()
    patent_cpc.rename(columns={"patent_id": "start_id", "cpc_class": "end_id"}, inplace=True)
    patent_cpc["type"] = "CLASSIFIED_AS"
    rels.append(("patent_cpc.csv", patent_cpc))

# Patent → Assignee (use organization name if ID missing)
if "disambig_assignee_organization" in df.columns:
    patent_assignee = df[["patent_id", "disambig_assignee_organization"]].dropna().drop_duplicates()
    patent_assignee.rename(columns={"patent_id": "start_id", "disambig_assignee_organization": "end_id"}, inplace=True)
    patent_assignee["type"] = "ASSIGNED_TO"
    rels.append(("patent_assignee.csv", patent_assignee))

# Patent → Inventor (if any)
if "disambig_inventor_name_last" in df.columns:
    df["inventor_name"] = (
        df[["disambig_inventor_name_first", "disambig_inventor_name_last"]]
        .fillna("")
        .agg(" ".join, axis=1)
        .str.strip()
    )
    patent_inventor = df[["patent_id", "inventor_name"]].dropna().drop_duplicates()
    patent_inventor.rename(columns={"patent_id": "start_id", "inventor_name": "end_id"}, inplace=True)
    patent_inventor["type"] = "PATENTED_BY"
    rels.append(("patent_inventor.csv", patent_inventor))

# Assignee → Location (use organization name and location_info)
if "assignee_location_info" in df.columns and "disambig_assignee_organization" in df.columns:
    ass_loc = df[["disambig_assignee_organization", "assignee_location_info"]].dropna().drop_duplicates()
    # Map textual location to generated ID
    loc_map = loc_nodes.set_index("location")["id"].to_dict()
    ass_loc["end_id"] = ass_loc["assignee_location_info"].map(loc_map)
    ass_loc.rename(columns={"disambig_assignee_organization": "start_id"}, inplace=True)
    ass_loc["type"] = "LOCATED_IN"
    ass_loc = ass_loc[["start_id", "end_id", "type"]]
    rels.append(("assignee_location.csv", ass_loc))

# Save all relationship files
for name, rel_df in rels:
    rel_df.to_csv(f"{OUT}/{name}", index=False)
    print(f"🔗 {name}: {len(rel_df):,} relationships")

print(f"\n✅ All Neo4j CSVs saved in → {OUT}")


🔗 patent_cpc.csv: 15,786 relationships
🔗 patent_assignee.csv: 15,442 relationships
🔗 patent_inventor.csv: 15,898 relationships
🔗 assignee_location.csv: 4,339 relationships

✅ All Neo4j CSVs saved in → /content/drive/MyDrive/Patent_Data/neo4j_exports_2024


In [ ]:
import pandas as pd
import re

ENRICHED_PATH = "/content/drive/MyDrive/Patent_Data/patents_2024_enriched_with_locations.parquet"
OUT_PATH = "/content/drive/MyDrive/Patent_Data/patents_2024_tech_final.parquet"

df = pd.read_parquet(ENRICHED_PATH)
print(f"Loaded {len(df):,} patents")

KEYWORDS = [
    "machine learning", "deep learning", "artificial intelligence",
    "neural network", "data pipeline", "data processing", "data management",
    "data storage", "software system", "computer system", "cloud computing",
    "distributed system", "reinforcement learning", "graph neural network",
    "data engineering", "algorithm optimization", "information retrieval",
    "recommendation system", "knowledge graph", "database system",
    "computational method"
]

pattern = re.compile("|".join(KEYWORDS), flags=re.IGNORECASE)

mask = (
    df["patent_title"].fillna("") + " " + df["patent_abstract"].fillna("")
).apply(lambda x: bool(pattern.search(x)))

tech_df = df.loc[mask].reset_index(drop=True)
tech_df.to_parquet(OUT_PATH, index=False)

print(f"✅ Filtered {len(tech_df):,} tech-related patents → {OUT_PATH}")



Loaded 2,616,964 patents
✅ Filtered 54,400 tech-related patents → /content/drive/MyDrive/Patent_Data/patents_2024_tech_final.parquet


In [ ]:
!pip -q install openai faiss-cpu pandas pyarrow tqdm tenacity numpy

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 23.6/23.6 MB 66.7 MB/s eta 0:00:00


In [ ]:
import os
from google.colab import userdata

os.environ["OPENAI_API_KEY"] = userdata.get('OPENAI_API_KEY')
print("✅ Loaded secret from Colab")

✅ Loaded secret from Colab


In [ ]:
from openai import OpenAI
from sentence_transformers import util
import faiss, numpy as np
import pandas as pd

client = OpenAI(api_key = os.getenv("OPENAI_API_KEY"))

# Load your enriched dataset
df = pd.read_parquet("/content/drive/MyDrive/Patent_Data/patents_2024_tech_final.parquet")

df.columns

Index(['patent_id', 'patent_date', 'patent_title', 'patent_abstract',
       'cpc_section', 'cpc_class', 'cpc_subclass', 'cpc_group', 'cpc_type',
       'disambig_inventor_name_first', 'disambig_inventor_name_last',
       'gender_code', 'disambig_assignee_individual_name_first',
       'disambig_assignee_individual_name_last',
       'disambig_assignee_organization', 'assignee_type',
       'assignee_location_info'],
      dtype='object')

In [ ]:
from openai import OpenAI
import pandas as pd
import numpy as np
import time, os

client = OpenAI(api_key=os.getenv("OPENAI_API_KEY"))

DATA_PATH = "/content/drive/MyDrive/Patent_Data/patents_2024_tech_final.parquet"
OUT_PATH = "/content/drive/MyDrive/Patent_Data/patents_2024_embeddings_tech.parquet"

BATCH_SIZE = 50   # safe for T4 and API rate limits
SLEEP_BETWEEN = 2 # seconds between batches

df = pd.read_parquet(DATA_PATH)
print(f"Loaded {len(df):,} patents")

# --- Create a unified search text ---


df["search_text"] = (
    "Title: " + df["patent_title"].fillna("") + ". " +
    "Abstract: " + df["patent_abstract"].fillna("") + ". " +
    "Assignee: " + df.get("disambig_assignee_organization", "").fillna("")+ ". " +
    "Location: " + df.get("assignee_location_info").fillna("")
)

# --- Generate embeddings in chunks ---
embeddings, indices = [], []

for start in range(0, len(df), BATCH_SIZE):
    end = min(start + BATCH_SIZE, len(df))
    batch = df["search_text"].iloc[start:end].tolist()

    try:
        resp = client.embeddings.create(
            model="text-embedding-3-large",
            input=batch
        )
        batch_emb = [np.array(d.embedding, dtype=np.float32) for d in resp.data]
        embeddings.extend(batch_emb)
        indices.extend(range(start, end))

        print(f"✅ Processed {end:,}/{len(df):,}")
        time.sleep(SLEEP_BETWEEN)

        # Periodic save
        if end % 500 == 0 or end == len(df):
            temp = df.iloc[:end].copy()
            temp["embedding"] = embeddings[:len(temp)]
            temp.to_parquet(OUT_PATH, index=False)
            print(f"💾 Saved progress ({end:,} records) → {OUT_PATH}")

    except Exception as e:
        print(f"⚠️ Error at {start}-{end}: {e}")
        time.sleep(10)
        continue

print(f"🎉 Done! Saved {len(embeddings):,} embeddings → {OUT_PATH}")


Loaded 54,400 patents
✅ Processed 50/54,400
✅ Processed 100/54,400
✅ Processed 150/54,400
✅ Processed 200/54,400
✅ Processed 250/54,400
✅ Processed 300/54,400
✅ Processed 350/54,400
✅ Processed 400/54,400
✅ Processed 450/54,400
✅ Processed 500/54,400
💾 Saved progress (500 records) → /content/drive/MyDrive/Patent_Data/patents_2024_embeddings_tech.parquet
✅ Processed 550/54,400
✅ Processed 600/54,400
✅ Processed 650/54,400
✅ Processed 700/54,400
✅ Processed 750/54,400
✅ Processed 800/54,400
✅ Processed 850/54,400
✅ Processed 900/54,400
✅ Processed 950/54,400
✅ Processed 1,000/54,400
💾 Saved progress (1,000 records) → /content/drive/MyDrive/Patent_Data/patents_2024_embeddings_tech.parquet
✅ Processed 1,050/54,400
✅ Processed 1,100/54,400
✅ Processed 1,150/54,400
✅ Processed 1,200/54,400
✅ Processed 1,250/54,400
✅ Processed 1,300/54,400
✅ Processed 1,350/54,400
✅ Processed 1,400/54,400
✅ Processed 1,450/54,400
✅ Processed 1,500/54,400
💾 Saved progress (1,500 records) → /content/drive/MyDri

KeyboardInterrupt: 

In [ ]:
import pandas as pd
import numpy as np
import faiss

# Load the saved embeddings
EMB_PATH = "/content/drive/MyDrive/Patent_Data/patents_2024_embeddings_tech.parquet"
df = pd.read_parquet(EMB_PATH)
print(f"Loaded {len(df):,} embeddings")

# Convert list objects → NumPy matrix
embeddings = np.vstack(df["embedding"].to_numpy())

# Create FAISS index (cosine similarity)
dim = embeddings.shape[1]
index = faiss.IndexFlatIP(dim)

# Normalize for cosine
faiss.normalize_L2(embeddings)
index.add(embeddings)

print(f"✅ FAISS index built with {index.ntotal:,} vectors")


Loaded 5,000 embeddings
✅ FAISS index built with 5,000 vectors


In [ ]:
# -------- Helper: safe lookup --------
def safe(row, col):
    return str(row[col]) if col in row and pd.notna(row[col]) else "Not Available"

# -------- Semantic search --------
def search_patents(query, top_k=5):
    q_emb = client.embeddings.create(model="text-embedding-3-large", input=query).data[0].embedding
    q_emb = np.array(q_emb, dtype=np.float32).reshape(1, -1)
    faiss.normalize_L2(q_emb)
    scores, idx = index.search(q_emb, top_k)
    results = df.iloc[idx[0]].copy()
    results["score"] = scores[0]
    return results

# -------- RAG generation --------
def rag_answer(query, context_df):
    context = "\n\n".join(
        f"Title: {r.patent_title}\nAbstract: {r.patent_abstract}"
        for _, r in context_df.iterrows()
    )
    prompt = f"""You are a patent assistant.
Use the following patent abstracts to answer the query concisely.

Query: {query}

Patents:
{context}
"""
    resp = client.chat.completions.create(
        model="gpt-4o-mini",
        messages=[{"role": "user", "content": prompt}],
        temperature=0.2
    )
    return resp.choices[0].message.content

# -------- Gradio callback --------
def search_ui(query):
    context_df = search_patents(query, top_k=5)
    summary = rag_answer(query, context_df)

    cards = ""
    for _, r in context_df.iterrows():
        cards += f"""
<div style='border:1px solid #ddd;border-radius:10px;padding:12px;margin-bottom:12px;
            box-shadow:0 1px 3px rgba(0,0,0,0.1);'>
  <h4>{safe(r, 'patent_title')}</h4>
  <p><b>Patent ID:</b> {safe(r, 'patent_id')}</p>
  <p><b>CPC Code:</b> {safe(r, 'cpc_group')}</p>
  <p><b>Assignee:</b> {safe(r, 'disambig_assignee_organization')}</p>
  <p><b>Publication Date:</b> {safe(r, 'patent_date')}</p>
  <p><b>Abstract:</b> {safe(r, 'patent_abstract')[:400]}...</p>
  <p><b>Similarity Score:</b> {r['score']:.3f}</p>
</div>
"""

    html = f"""
<h2>🔍 Query:</h2><p>{query}</p>
<h3>🧠 Answer Summary:</h3><p>{summary}</p>
<hr>
<h3>📄 Top Matching Patents:</h3>
{cards}
"""
    return html

# -------- Launch Gradio --------
import gradio as gr
gr.Interface(
    fn=search_ui,
    inputs=gr.Textbox(label="Enter a technical query or keyword"),
    outputs=gr.HTML(),
    title="🔬 Patent Semantic Search RAG System",
    description="Search across 2024 technology patents and retrieve the most relevant results with contextual answers."
).launch()

It looks like you are running Gradio on a hosted Jupyter notebook, which requires `share=True`. Automatically setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://f397043ee599ee9143.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)


In [ ]:
from openai import OpenAI

client = OpenAI(api_key=os.getenv("OPENAI_API_KEY"))

def search_patents(query, top_k=5):
    # Encode query
    q_emb = client.embeddings.create(
        model="text-embedding-3-large",
        input=query
    ).data[0].embedding
    q_emb = np.array(q_emb, dtype=np.float32).reshape(1, -1)
    faiss.normalize_L2(q_emb)

    # Search
    scores, idx = index.search(q_emb, top_k)
    results = df.iloc[idx[0]].copy()
    results["score"] = scores[0]
    return results[["patent_id","patent_title","patent_abstract","disambig_assignee_organization", "assignee_location_info","score"]]


In [ ]:
search_patents("distributed data pipeline optimization")


,patent_id,patent_title,patent_abstract,disambig_assignee_organization,assignee_location_info,score
79,11860877,Streaming data analytics using data pipelines ...,One example method includes receiving a data s...,Dell Products L.P.,"Round Rock, TX, US",0.507518
64,11860820,Processing data through a storage system in a ...,Processing data through a storage system in a ...,"Pure Storage, Inc.","Santa Clara, CA, US",0.448874
135,11861331,Scaling high-level statistical languages to la...,A system and method for performing large-scale...,Google LLC,"Mountain View, CA, US",0.424981
342,11863675,Data flow control in distributed computing sys...,Datacenters or other large-scale distributed c...,"Microsoft Technology Licensing, LLC","Redmond, WA, US",0.402695
320,11863398,Centralized management of distributed data sou...,Aspects of the disclosure are directed to a ce...,Google LLC,"Mountain View, CA, US",0.395015


In [ ]:
def rag_answer(query):
    context_df = search_patents(query, top_k=5)
    context = "\n\n".join(
        f"Title: {row.patent_title}\nAbstract: {row.patent_abstract}\nOrganization:{row.disambig_assignee_organization}\nLocation:{row.assignee_location_info}"
        for _, row in context_df.iterrows()
    )

    prompt = f"""You are a patent assistant.
Use the following patent information like title, abstract, organization and location to answer the query concisely.

Query: {query}

Patents:
{context}
"""
    response = client.chat.completions.create(
        model="gpt-4o-mini",
        messages=[{"role":"user","content":prompt}],
        temperature=0.2
    )
    return response.choices[0].message.content


In [ ]:
rag_answer("new approaches to data pipeline optimization")


'Here are some patents related to new approaches to data pipeline optimization:\n\n1. **Optimizing flow of data within ETL data processing pipeline**\n   - Organization: International Business Machines Corporation\n   - Location: Armonk, NY, US\n\n2. **Streaming data analytics using data pipelines and knowledge graphs**\n   - Abstract: This method involves receiving a real-time data stream, generating metadata, caching portions of the data, and analyzing the cached data to generate insights.\n   - Organization: Dell Products L.P.\n   - Location: Round Rock, TX, US\n\n3. **Cross-organization and cross-cloud automated data pipelines**\n   - Organization: Snowflake Inc.\n   - Location: Bozeman, MT, US\n\n4. **Creation and management of data pipelines**\n   - Organization: Calibo LLC\n   - Location: Miami, FL, US\n\n5. **Data pipeline error detection**\n   - Organization: StreamSets, Inc.\n   - Location: San Mateo, CA, US\n\nThese patents reflect various innovative approaches to optimizing

In [ ]:
import gradio as gr

def search_ui(query):
    df_results = search_patents(query, top_k=5)
    response = rag_answer(query)
    cards = "\n\n".join(
        f"**{r.patent_title}**\n\n{r.patent_abstract[:500]}..."
        for _, r in df_results.iterrows()
    )
    return f"### 🔍 Query: {query}\n\n{response}\n\n---\n{cards}"

gr.Interface(
    fn=search_ui,
    inputs="text",
    outputs="markdown",
    title="🔬 Patent Semantic Search (RAG Prototype)",
    description="Ask a question or describe a technology to explore related 2024 patents."
).launch()


It looks like you are running Gradio on a hosted Jupyter notebook, which requires `share=True`. Automatically setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://99e23d50e125c0e0c4.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)
